In [3]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import json
import folium
from sklearn.cluster import KMeans, DBSCAN

In [4]:
# Read votation data
df = pd.read_pickle("data/data.pkl")

In [5]:
df.head()

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,Canton,Pays
0,Aeugst am Albis,29.11.1998 Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,Zürich,Suisse
1,Aeugst am Albis,14.06.2015 Initiative sur les bourses d'études,1380.0,706.0,51.2,695.0,186.0,509.0,26.8,Affoltern,Zürich,Suisse
2,Aeugst am Albis,25.09.2016 Loi fédérale sur le renseignement,1400.0,670.0,47.9,659.0,417.0,242.0,63.3,Affoltern,Zürich,Suisse
3,Aeugst am Albis,03.03.1991 Encouragement des transports publics,835.0,321.0,38.4,312.0,128.0,184.0,41.0,Affoltern,Zürich,Suisse
4,Aeugst am Albis,12.02.2017 Réforme de l'imposition des entrepr...,1395.0,759.0,54.4,750.0,318.0,432.0,42.4,Affoltern,Zürich,Suisse


In [6]:
commune_list = list(set(df['Commune'].values))
print(len(commune_list))
commune_dict = { val : idx for idx , val in enumerate(commune_list)   }

votation_list = list(set(df['Votation'].values))
votation_dict = { val : idx for idx , val in enumerate(votation_list)   }

X = np.ones((len(commune_list) , len(votation_list) ) , dtype=float)

2165


In [7]:
for x in df [['Commune','Votation','Oui en %']].fillna(50).values :
    X [commune_dict[x[0]]][votation_dict[x[1]]] = x[2]

In [8]:
X

array([[  43.8,   18.9,   18.9, ...,   18.1,   19.5,   90.3],
       [  39.6,   20. ,   30. , ...,    7.5,   35.7,  100. ],
       [  38.8,   16.1,   21.2, ...,   22.3,   10.8,   88.4],
       ..., 
       [  43.7,   32.7,   26.1, ...,   18. ,   23.6,   90.1],
       [  32.4,   13.3,   24.2, ...,   37.2,   28.8,   87.6],
       [  46.4,   41.4,   44.9, ...,   28.1,   70.6,   97.6]])

In [9]:
def get_good_date (x) :
    date = x.split(' ') [0]
    return '.'.join(date.split('.')[::-1])


#return -1 if date1 < date2 return 1 if date1 > date2 return 0 if if date1 == date2
def compare_date (date1 , date2) :
    date1_s = date1.split('.')
    date2_s = date2.split('.')
    for i in range ( len(date1_s)) :
        if ( int(date1_s[i]) < int(date2_s[i])) :
            return -1
        elif ( int(date1_s[i]) > int(date2_s[i])) :
            return 1
    return 0



df['Date'] = df['Votation'].map( get_good_date)
df.head()

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,Canton,Pays,Date
0,Aeugst am Albis,29.11.1998 Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,Zürich,Suisse,1998.11.29
1,Aeugst am Albis,14.06.2015 Initiative sur les bourses d'études,1380.0,706.0,51.2,695.0,186.0,509.0,26.8,Affoltern,Zürich,Suisse,2015.06.14
2,Aeugst am Albis,25.09.2016 Loi fédérale sur le renseignement,1400.0,670.0,47.9,659.0,417.0,242.0,63.3,Affoltern,Zürich,Suisse,2016.09.25
3,Aeugst am Albis,03.03.1991 Encouragement des transports publics,835.0,321.0,38.4,312.0,128.0,184.0,41.0,Affoltern,Zürich,Suisse,1991.03.03
4,Aeugst am Albis,12.02.2017 Réforme de l'imposition des entrepr...,1395.0,759.0,54.4,750.0,318.0,432.0,42.4,Affoltern,Zürich,Suisse,2017.02.12


In [10]:
test = df [ df ['Date'].map (lambda x : compare_date ( '2000.00.00' , x ) == -1 )]
test

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,Canton,Pays,Date
1,Aeugst am Albis,14.06.2015 Initiative sur les bourses d'études,1380.0,706.0,51.2,695.0,186.0,509.0,26.8,Affoltern,Zürich,Suisse,2015.06.14
2,Aeugst am Albis,25.09.2016 Loi fédérale sur le renseignement,1400.0,670.0,47.9,659.0,417.0,242.0,63.3,Affoltern,Zürich,Suisse,2016.09.25
4,Aeugst am Albis,12.02.2017 Réforme de l'imposition des entrepr...,1395.0,759.0,54.4,750.0,318.0,432.0,42.4,Affoltern,Zürich,Suisse,2017.02.12
5,Aeugst am Albis,07.03.2010 Recherche sur l'être humain,1262.0,588.0,46.6,579.0,476.0,103.0,82.2,Affoltern,Zürich,Suisse,2010.03.07
6,Aeugst am Albis,08.02.2004 Initiative pour l'internement des d...,1135.0,590.0,52.0,578.0,266.0,312.0,46.0,Affoltern,Zürich,Suisse,2004.02.08
7,Aeugst am Albis,"22.09.2002 L'or à l'AVS, aux cantons et à la F...",1113.0,610.0,54.8,604.0,325.0,267.0,54.9,Affoltern,Zürich,Suisse,2002.09.22
8,Aeugst am Albis,24.11.2002 Contre les abus dans le droit d'asile,1117.0,662.0,59.3,648.0,282.0,366.0,43.5,Affoltern,Zürich,Suisse,2002.11.24
13,Aeugst am Albis,26.11.2000 Initiative «pour des coûts hospital...,1120.0,456.0,40.7,449.0,48.0,401.0,10.7,Affoltern,Zürich,Suisse,2000.11.26
16,Aeugst am Albis,09.02.2014 Initiative «Financer l'avortement e...,1374.0,857.0,62.4,849.0,290.0,559.0,34.2,Affoltern,Zürich,Suisse,2014.02.09
18,Aeugst am Albis,07.03.2010 Initiative pour l'institution d'un ...,1262.0,599.0,47.5,592.0,246.0,346.0,41.6,Affoltern,Zürich,Suisse,2010.03.07


In [11]:
switzerland_coord = [46.765213, 8.252444]
town_geo_path = r'data/switzerland_borders/admin_level_8.geojson'
geo_json_data = json.load(open(town_geo_path, encoding="utf8"))

In [52]:
color_list = ['#ff0000' , '#00ff00' , '#0000ff' , '#ffff00' , '#ff00ff' , '#00ffff'  , '#000000']


In [16]:
def style_function(feature):
    #print(feature['name'])
    language = languages_series.get(feature['name'], None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor':  color_language(language)
    }

def draw_map_kmeans (n_clusters , X ) :
    kmeans_res = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
    groups = kmeans_res.labels_
    
    centers = kmeans_res.cluster_centers_
    print( [ [ np.linalg.norm(x-y) for y in centers] for x in centers])
    
    to_map = pd.DataFrame({'Commune' : commune_list , 'Group' : groups})
    
    map1 = folium.Map(location=switzerland_coord, zoom_start=8)
    map1.choropleth(geo_data = geo_json_data, \
                                    data = to_map, \
                                    columns = ['Commune', 'Group'], \
                                    key_on = 'feature.name', \
                                    fill_color = 'RdYlGn', \
                                     fill_opacity = 0.7, \
                                    line_opacity = 0.2, \
                                    legend_name = 'group')
    
    return map1

In [54]:
for i in range (2,6) :  
    draw_map_kmeans(i,X).save('data/map_ml/kmeans'+str(i)+'.html')

[[0.0, 208.67401237835256], [208.67401237835256, 0.0]]
[[0.0, 247.21580842824966, 196.3873395166566], [247.21580842824966, 0.0, 144.46503224810519], [196.3873395166566, 144.46503224810519, 0.0]]
[[0.0, 221.59382027546039, 236.36037086236314, 193.87074264647026], [221.59382027546039, 0.0, 247.64281574780981, 207.85672978130717], [236.36037086236314, 247.64281574780981, 0.0, 141.34595602506744], [193.87074264647026, 207.85672978130717, 141.34595602506744, 0.0]]
[[0.0, 237.04607787057085, 242.72212200864692, 214.700838379114, 194.22622625708948], [237.04607787057085, 0.0, 275.89090092058183, 222.98561992569219, 140.64525255314484], [242.72212200864692, 275.89090092058183, 0.0, 141.06638785088916, 215.67520469036879], [214.700838379114, 222.98561992569219, 141.06638785088916, 0.0, 221.63966288565695], [194.22622625708948, 140.64525255314484, 215.67520469036879, 221.63966288565695, 0.0]]


In [55]:
def draw_map_DBSCAN (X) : 
    min_samples = 20

    X_array = [ np.array(x_) for x_ in X]
    range_X = range(len(X))
    Xmeans = np.mean([ np.mean(\
                            np.sort([np.linalg.norm(X_array[x]-X_array[y]) \
                             for x in range_X  if x!=y])[:(min_samples-1)] \
                           )\
                   for y in range_X ] )
    groups =  DBSCAN(eps=Xmeans, min_samples=min_samples).fit(X).labels_

    
  
    
    to_map = pd.DataFrame({'Commune' : commune_list , 'Group' : groups})
    
    map1 = folium.Map(location=switzerland_coord, zoom_start=8)
    map1.choropleth(geo_data = geo_json_data, \
                                    data = to_map, \
                                    columns = ['Commune', 'Group'], \
                                    key_on = 'feature.name', \
                                    fill_color = 'RdYlGn', \
                                     fill_opacity = 0.7, \
                                    line_opacity = 0.2, \
                                    legend_name = 'group')
    
    return map1

In [56]:
draw_map_DBSCAN (X).save('data/map_ml/DBSCAN.html')

In [57]:
import networkx as nx
G = nx.DiGraph()

min_max =[0,0]
max_value = -1


X_len = len(X)
for i in range (X_len) :
    #G.add_edge('g',i , capacity =10^3000)
    #G.add_edge(i,'e' , capacity = 10^3000)
    for j in range (i+1,X_len) :
        dist = 1-np.linalg.norm(X[i]/100-X[j]/100)
        G.add_edge(i,j, capacity = dist)
        G.add_edge(j,i, capacity = dist)
        if ( dist > max_value) :
            min_max = [i,j]
G

In [58]:
cut_value, partition = nx.minimum_cut(G, min_max[0],min_max[1])
reachable, non_reachable = partition

In [59]:
cut_value

1.591151849379266

In [60]:
len(reachable)


2116

In [61]:
G = nx.DiGraph()
G.add_edge('x','a', capacity = 3.0)
G.add_edge('x','b', capacity = 1.0)
G.add_edge('a','c', capacity = 3.0)
G.add_edge('b','c', capacity = 5.0)
G.add_edge('b','d', capacity = 4.0)
G.add_edge('d','e', capacity = 2.0)
G.add_edge('c','y', capacity = 2.0)
G.add_edge('e','y', capacity = 3.0)


cut_value, partition = nx.minimum_cut(G, 'x', 'y')
reachable, non_reachable = partition
reachable

{'a', 'c', 'x'}

ml for theme

In [12]:
data_t = pd.read_pickle('data/data_theme.pkl')
data_t.head()

,Votation,Thématique
0,14.06.1981 Protection des consommateurs,Economie
1,29.11.1981 Régime financier,Finances publiques
2,14.06.1981 Egalité entre hommes et femmes,Politique sociale
3,06.06.1982 Code pénal suisse,Régime politique
4,06.06.1982 Loi sur les étrangers,Politique sociale


In [13]:
df.head()

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,Canton,Pays,Date
0,Aeugst am Albis,29.11.1998 Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,Zürich,Suisse,1998.11.29
1,Aeugst am Albis,14.06.2015 Initiative sur les bourses d'études,1380.0,706.0,51.2,695.0,186.0,509.0,26.8,Affoltern,Zürich,Suisse,2015.06.14
2,Aeugst am Albis,25.09.2016 Loi fédérale sur le renseignement,1400.0,670.0,47.9,659.0,417.0,242.0,63.3,Affoltern,Zürich,Suisse,2016.09.25
3,Aeugst am Albis,03.03.1991 Encouragement des transports publics,835.0,321.0,38.4,312.0,128.0,184.0,41.0,Affoltern,Zürich,Suisse,1991.03.03
4,Aeugst am Albis,12.02.2017 Réforme de l'imposition des entrepr...,1395.0,759.0,54.4,750.0,318.0,432.0,42.4,Affoltern,Zürich,Suisse,2017.02.12


In [14]:
data_theme = df.merge(data_t , on = 'Votation' )
data_theme.head()

,Commune,Votation,Electeurs inscrits,Bulletins rentrés,Participation en %,Bulletins valables,Oui,Non,Oui en %,District,Canton,Pays,Date,Thématique
0,Aeugst am Albis,29.11.1998 Initiative Droleg,1070.0,487.0,45.5,478.0,167.0,311.0,34.9,Affoltern,Zürich,Suisse,1998.11.29,Politique sociale
1,Affoltern am Albis,29.11.1998 Initiative Droleg,5729.0,2286.0,39.9,2236.0,678.0,1558.0,30.3,Affoltern,Zürich,Suisse,1998.11.29,Politique sociale
2,Bonstetten,29.11.1998 Initiative Droleg,2596.0,1063.0,40.9,1045.0,358.0,687.0,34.3,Affoltern,Zürich,Suisse,1998.11.29,Politique sociale
3,Hausen am Albis,29.11.1998 Initiative Droleg,2081.0,807.0,38.8,792.0,281.0,511.0,35.5,Affoltern,Zürich,Suisse,1998.11.29,Politique sociale
4,Hedingen,29.11.1998 Initiative Droleg,1858.0,810.0,43.6,791.0,246.0,545.0,31.1,Affoltern,Zürich,Suisse,1998.11.29,Politique sociale


In [ ]:
#data_bg = data by group
for data_bg in data_theme[['Thématique','Votation','Commune','Oui en %']].groupby('Thématique') :
    theme = data_bg[0]
    data_bg = pd.DataFrame(data = data_bg[1])

    votation_list_t = list(set(data_bg['Votation'].values))
    votation_dict_t = { val : idx for idx , val in enumerate(votation_list_t)   }

    Xt = np.ones((len(commune_list) , len(votation_list_t) ) , dtype=float)
    
    for x in data_bg [['Commune','Votation','Oui en %']].fillna(50).values :
        Xt [commune_dict[x[0]]][votation_dict_t[x[1]]] = x[2]
    draw_map_kmeans(2,Xt).save('data/maps_theme_ml/kmeans_'+theme+'.html') 
    

[[0.0, 68.555355791529735], [68.555355791529735, 0.0]]
[[0.0, 58.053538348416119], [58.053538348416119, 0.0]]
[[0.0, 41.901041386400706], [41.901041386400706, 0.0]]
[[0.0, 102.52886424138821], [102.52886424138821, 0.0]]
